In [4]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from preprocessing_pipeline import create_preprocessing_pipeline

# Data Loading
Load the dataset and show first few rows. 

In [ ]:
# Load the dataset
file_path = './marketing_campaign.csv'
df = pd.read_csv(file_path, sep='\t')
df.head()

# Create a Master Pipeline
 Chain data preprocessing and model steps together in a single object to ensure that all steps are applied sequentially and that data leakage is avoided.

In [6]:
# Create the pipeline with preprocessing and KMeans
master_pipeline = Pipeline(steps=[
    ('preprocessing', create_preprocessing_pipeline()),
    ('kmeans', KMeans())
])

master_pipeline

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('feature_engineering',
                                  FunctionTransformer(func=<function create_preprocessing_pipeline.<locals>.<lambda> at 0x000002865CE0CEE0>)),
                                 ('preprocessing',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['...
                                                                    'NumDealsPurchases',
                                                                    'NumWebPurchases',
                                                                    'NumCatalogPurchases',
                                                                    'NumStorePurchases',
                                                                    'NumWebVisitsMonth',
                                                                    'Z_CostContact',
                                                                    'Z_Revenue',
                                                                    'Age',
                                                                    'Total_Children',
                                                                    'Total_Household_Size',
                                                                    'Total_Spending',
                                                                    'Loyalty_Months',
                                                                    'Campaign_Response_Rate',
                                                                    'Shopping_Frequency']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['Education',
                                                                    'Marital_Status_Grouped'])]))])),
                ('kmeans', KMeans())])

# Tune hyperparameters and find clusters
- Use elbow method to find k value
- Run the pipeline with the optimal value of k 
- Label the clusters and visualize it